In [1]:
import firebase_admin
import google.cloud
from firebase_admin import credentials, firestore

In [2]:
import pytz
from datetime import datetime, tzinfo, date
from pprint import pprint as pp

In [3]:
cred = credentials.Certificate('mentor_connect_private.json')
firebase_admin.initialize_app(cred)

In [4]:
db = firestore.client()

In [71]:
u_ref = db.collection(u'users')
u_ref

In [80]:
users = u_ref.stream()
users

<generator object Query.stream at 0x000001C964905648>

In [81]:
for r in users:
    print(r.id)
    user = r.to_dict()
#     if 'Date' in user:
#         user['Date'] = user['Date'].replace(tzinfo=pytz.timezone('US/Pacific'))
    print(user)

100
{'course': 'data structures', 'name': 'Santhiya'}
101
{'course': 'anatomy', 'name': 'Divya'}
102
{'course': None, 'name': 'Dilip'}


In [61]:
u_ref.add({
    'Name': 'Burhan3',
    "Course": 'CS',
    "date_test": datetime.now(pytz.timezone('US/Pacific'))
})

(seconds: 1580560639
 nanos: 919147000,
 <google.cloud.firestore_v1.document.DocumentReference at 0x1c964bc6c08>)

In [23]:
help(u_ref.add)

Help on method add in module google.cloud.firestore_v1.collection:

add(document_data, document_id=None) method of google.cloud.firestore_v1.collection.CollectionReference instance
    Create a document in the Firestore database with the provided data.
    
    Args:
        document_data (dict): Property names and values to use for
            creating the document.
        document_id (Optional[str]): The document identifier within the
            current collection. If not provided, an ID will be
            automatically assigned by the server (the assigned ID will be
            a random 20 character string composed of digits,
            uppercase and lowercase letters).
    
    Returns:
        Tuple[:class:`google.protobuf.timestamp_pb2.Timestamp`,                 :class:`~google.cloud.firestore_v1.document.DocumentReference`]:
            Pair of
    
            * The ``update_time`` when the document was created/overwritten.
            * A document reference for the create

In [105]:
help(db.collections)
x = db.collections()

Help on method collections in module google.cloud.firestore_v1.client:

collections() method of google.cloud.firestore_v1.client.Client instance
    List top-level collections of the client's database.
    
    Returns:
        Sequence[:class:`~google.cloud.firestore_v1.collection.CollectionReference`]:
            iterator of subcollections of the current document.



In [148]:
userID = '101'
transactions = db.collection(u'transactions').where('tutor_id', '==', userID).stream()
for t in transactions:
    tdict = t.to_dict()
    cdict = db.collection('courses').document(str(tdict['course_id'])).get().to_dict()
    sdict = db.collection('subjects').document(str(cdict['subject_id'])).get().to_dict()
    student_dict = db.collection('users').document(str(tdict['student_id'])).get().to_dict()
    tutor_dict = db.collection('users').document(str(tdict['tutor_id'])).get().to_dict()
    
    help(tdict['appointment'])
    break
    tdict['appointment'] = tdict['appointment'].astimezone(pytz.timezone('US/Pacific')).isoformat()
    cdict['subject'] = sdict
    tdict['course'] = cdict
    tdict['student'] = student_dict
    tdict['tutor'] = tutor_dict
    pp(tdict)


Help on DatetimeWithNanoseconds in module google.api_core.datetime_helpers object:

class DatetimeWithNanoseconds(datetime.datetime)
 |  DatetimeWithNanoseconds(*args, **kw)
 |  
 |  Track nanosecond in addition to normal datetime attrs.
 |  
 |  Nanosecond can be passed only as a keyword argument.
 |  
 |  Method resolution order:
 |      DatetimeWithNanoseconds
 |      datetime.datetime
 |      datetime.date
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  rfc3339(self)
 |      Return an RFC3339-compliant timestamp.
 |      
 |      Returns:
 |          (str): Timestamp string according to RFC3339 spec.
 |  
 |  timestamp_pb(self)
 |      Return a timestamp message.
 |      
 |      Returns:
 |          (:class:`~google.protobuf.timestamp_pb2.Timestamp`): Timestamp message
 |  
 |  ----------------------------------------------------------------------
 |  Class methods defined here:
 |  
 |  from_rfc3339(stamp) from builtins.type
 |      Parse RFC3339-compliant timesta

In [84]:
users = u_ref.stream()
users

<generator object Query.stream at 0x000001C964A67A48>

In [85]:
for r in users:
    print(r.id)
    user = r.to_dict()
#     if 'Date' in user:
#         user['Date'] = user['Date'].replace(tzinfo=pytz.timezone('US/Pacific'))
    print(user)

201
{'course_id': 1001, 'student_id': 102, 'tutor_id': 101, 'appointment': DatetimeWithNanoseconds(2020, 2, 1, 13, 5, tzinfo=<UTC>)}


In [140]:
transactions_iter = db.collection(u'transactions').stream()
transactions = []
tutor_set = set()
for t in transactions_iter:
    tdict = t.to_dict()
    tdict['tutor'] = db.collection('users').document(str(tdict['tutor_id'])).get().to_dict()
    tdict['appointment'] = tdict['appointment'].astimezone(pytz.timezone('US/Pacific')).isoformat()
    transactions.append(tdict)
    
transactions

[{'course_id': 1001,
  'student_id': 102,
  'tutor_id': 101,
  'appointment': '2020-02-01T05:05:00-08:00',
  'tutor': {'name': 'Divya'}},
 {'course_id': 1002,
  'student_id': 103,
  'tutor_id': 100,
  'appointment': '2020-02-06T12:30:00-08:00',
  'tutor': {'name': 'Santhiya'}},
 {'course_id': 1001,
  'student_id': 103,
  'tutor_id': 101,
  'appointment': '2020-02-24T12:12:00-08:00',
  'tutor': {'name': 'Divya'}},
 {'course_id': 1001,
  'student_id': 103,
  'tutor_id': 100,
  'appointment': '2020-02-12T13:45:00-08:00',
  'tutor': {'name': 'Santhiya'}}]

In [143]:
tid = 'dw4G26wfWqoeAVdRgiEy'
status = True

trans_ref = db.collection('transactions')
tdict = trans_ref.document(tid).get().to_dict()

trans_ref.document(tid).set({
    'status': 1 if status else 2
}, merge=True)

if not status:
    trans_ref.add(tdict)

In [11]:
user_iter = db.collection('users').where('username', '==', 'divya2000').stream()
help(user_iter)

Help on generator object:

stream = class generator(object)
 |  Methods defined here:
 |  
 |  __del__(...)
 |  
 |  __getattribute__(self, name, /)
 |      Return getattr(self, name).
 |  
 |  __iter__(self, /)
 |      Implement iter(self).
 |  
 |  __next__(self, /)
 |      Implement next(self).
 |  
 |  __repr__(self, /)
 |      Return repr(self).
 |  
 |  close(...)
 |      close() -> raise GeneratorExit inside generator.
 |  
 |  send(...)
 |      send(arg) -> send 'arg' into generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  throw(...)
 |      throw(typ[,val[,tb]]) -> raise exception in generator,
 |      return next yielded value or raise StopIteration.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  gi_code
 |  
 |  gi_frame
 |  
 |  gi_running
 |  
 |  gi_yieldfrom
 |      object being iterated by yield from, or None



In [10]:
for u in user_iter:
    print(u.to_dict())

{'phone': '+19495617353', 'username': 'divya2000', 'name': 'Divya', 'pay': '$ 18/hr', 'bio': 'I have got a close to perfect score during my highschool in bio! Have experience in 1-1.'}
